# Install pyspark library

In [36]:
!pip install pyspark

In [37]:
#import pyspark library
import pyspark

In [38]:
#import spark session library
from pyspark.sql import SparkSession

In [39]:
# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('random_forest_regression') \
                    .getOrCreate()

#  Create DataFrame

In [40]:
#To create dataframe form External datasets
df = spark.read.option("header", "true").csv("/Users/krishnapratap/Desktop/partation/data/*")

In [41]:
df.show()

+----+-------+-----+----------+---------+----------+-------------+---------+-------+-------+---------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+---------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+--------------

22/04/26 14:43:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, UniqueCarrier, AirlineID, Carrier, TailNum, FlightNum, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandings, DivReachedDest, DivActualElapsedTime, DivArrDelay, DivDistan

# Data Preparation

In [42]:
#create new dataframe as per required columns for prediction
AirlineDF = df.select("Origin","Dest","AirTime","Distance")

In [43]:
#cache data in-memory for fast read write operation
AirlineDF.cache()

DataFrame[Origin: string, Dest: string, AirTime: string, Distance: string]

In [44]:
#check data in new dataframe
AirlineDF.show()

ce, Div1Airport, Div1AirportID, Div1AirportSeqID, Div1WheelsOn, Div1TotalGTime, Div1LongestGTime, Div1WheelsOff, Div1TailNum, Div2Airport, Div2AirportID, Div2AirportSeqID, Div2WheelsOn, Div2TotalGTime, Div2LongestGTime, Div2WheelsOff, Div2TailNum, Div3Airport, Div3AirportID, Div3AirportSeqID, Div3WheelsOn, Div3TotalGTime, Div3LongestGTime, Div3WheelsOff, Div3TailNum, Div4Airport, Div4AirportID, Div4AirportSeqID, Div4WheelsOn, Div4TotalGTime, Div4LongestGTime, Div4WheelsOff, Div4TailNum, Div5Airport, Div5AirportID, Div5AirportSeqID, Div5WheelsOn, Div5TotalGTime, Div5LongestGTime, Div5WheelsOff, Div5TailNum, 
 Schema: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, UniqueCarrier, AirlineID, Carrier, TailNum, FlightNum, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSD

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX| 338.00| 2475.00|
|   JFK| LAX| 349.00| 2475.00|
|   JFK| LAX| 370.00| 2475.00|
|   JFK| LAX| 350.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 336.00| 2475.00|
|   JFK| LAX| 380.00| 2475.00|
|   JFK| LAX| 359.00| 2475.00|
|   JFK| LAX| 368.00| 2475.00|
|   JFK| LAX| 356.00| 2475.00|
|   JFK| LAX| 353.00| 2475.00|
|   JFK| LAX| 332.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 340.00| 2475.00|
|   JFK| LAX| 327.00| 2475.00|
|   JFK| LAX| 308.00| 2475.00|
|   JFK| LAX| 315.00| 2475.00|
|   JFK| LAX| 323.00| 2475.00|
+------+----+-------+--------+
only showing top 20 rows



In [45]:
#check data types of each columns
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)



# We need to change data types of columns

In [46]:
from pyspark.sql.types import IntegerType

In [47]:
AirlineDF = AirlineDF.withColumn("Distance", AirlineDF["Distance"].cast(IntegerType()))

In [48]:
AirlineDF = AirlineDF.withColumn("AirTime", AirlineDF["AirTime"].cast(IntegerType()))

In [49]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: integer (nullable = true)



# Check is there any null values in DataFrame 

In [50]:
#import libray for sql function col
from pyspark.sql.functions import col

In [51]:
#check null values in each columns
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())

0
0


183789


0


# we need to handel null values 

In [52]:
#now drop row as correspondence to null values
AirlineDF = AirlineDF.dropna(subset=["AirTime"])

In [53]:
#check again
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())

0
0


0


0


# we need convert distance miles into km

In [54]:
from pyspark.sql.functions import round

# Convert 'mile' to 'km' 
AirlineDF = AirlineDF.withColumn('Distance', round(AirlineDF.Distance * 1.60934, 0))

AirlineDF.show()

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX|    338|  3983.0|
|   JFK| LAX|    349|  3983.0|
|   JFK| LAX|    370|  3983.0|
|   JFK| LAX|    350|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    336|  3983.0|
|   JFK| LAX|    380|  3983.0|
|   JFK| LAX|    359|  3983.0|
|   JFK| LAX|    368|  3983.0|
|   JFK| LAX|    356|  3983.0|
|   JFK| LAX|    353|  3983.0|
|   JFK| LAX|    332|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    340|  3983.0|
|   JFK| LAX|    327|  3983.0|
|   JFK| LAX|    308|  3983.0|
|   JFK| LAX|    315|  3983.0|
|   JFK| LAX|    323|  3983.0|
+------+----+-------+--------+
only showing top 20 rows



In [55]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: double (nullable = true)



## Verctorize the features

In [56]:
from pyspark.ml.feature import *

In [57]:
from pyspark.ml.feature import VectorAssembler

In [58]:
vectorizer = VectorAssembler()
vectorizer.setInputCols(["Distance"])
vectorizer.setOutputCol("features")

df_vect = vectorizer.transform(AirlineDF)

In [59]:
df_vect.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   JFK| LAX|    338|  3983.0|[3983.0]|
|   JFK| LAX|    349|  3983.0|[3983.0]|
|   JFK| LAX|    370|  3983.0|[3983.0]|
|   JFK| LAX|    350|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    336|  3983.0|[3983.0]|
|   JFK| LAX|    380|  3983.0|[3983.0]|
|   JFK| LAX|    359|  3983.0|[3983.0]|
|   JFK| LAX|    368|  3983.0|[3983.0]|
|   JFK| LAX|    356|  3983.0|[3983.0]|
|   JFK| LAX|    353|  3983.0|[3983.0]|
|   JFK| LAX|    332|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    340|  3983.0|[3983.0]|
|   JFK| LAX|    327|  3983.0|[3983.0]|
|   JFK| LAX|    308|  3983.0|[3983.0]|
|   JFK| LAX|    315|  3983.0|[3983.0]|
|   JFK| LAX|    323|  3983.0|[3983.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



In [60]:
print(vectorizer.explainParams())

handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error)
inputCols: input column names. (current: ['Distance'])
outputCol: output column name. (default: VectorAssembler_7c41b79746bb__output, current: features)


# Train Test data Splitting

In [61]:
flights_train, flights_test = df_vect.randomSplit([0.8, 0.2])

In [62]:
flights_train.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   ABE| CLT|     70|   774.0| [774.0]|
|   ABE| CLT|     74|   774.0| [774.0]|
|   ABE| CLT|     76|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     78|   774.0| [774.0]|
|   ABE| CLT|     79|   774.0| [774.0]|
|   ABE| CLT|     79|   774.0| [774.0]|
|   ABE| CLT|     82|   774.0| [774.0]|
|   ABE| CLT|     82|   774.0| [774.0]|
|   ABE| CLT|     83|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     86|   774.0| [774.0]|
|   ABE| CLT|     87|   774.0| [774.0]|
|   ABE| CLT|     87|   774.0| [774.0]|
|   ABE| CLT|     87|   774.0| [774.0]|
|   ABE| CLT|     88|   774.0| [774.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



# RandonForest Regression Model Training

In [63]:
from pyspark.ml.regression import RandomForestRegressor

In [64]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features")
print(rf.explainParams())

bootstrap: Whether bootstrap samples are used when building trees. (default: True)
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the featur

In [65]:
rf.setLabelCol("AirTime")
rf.setFeaturesCol("features")
model = rf.fit(flights_train)

22/04/26 14:46:31 WARN MemoryStore: Not enough space to cache rdd_263_1 in memory! (computed 29.6 MiB so far)
22/04/26 14:46:31 WARN BlockManager: Persisting block rdd_263_1 to disk instead.
22/04/26 14:46:39 WARN MemoryStore: Not enough space to cache rdd_263_4 in memory! (computed 19.7 MiB so far)
22/04/26 14:46:39 WARN BlockManager: Persisting block rdd_263_4 to disk instead.
22/04/26 14:47:41 WARN MemoryStore: Not enough space to cache rdd_263_2 in memory! (computed 19.7 MiB so far)
22/04/26 14:47:41 WARN MemoryStore: Not enough space to cache rdd_263_3 in memory! (computed 19.7 MiB so far)
22/04/26 14:47:41 WARN MemoryStore: Not enough space to cache rdd_263_1 in memory! (computed 19.7 MiB so far)
22/04/26 14:47:41 WARN MemoryStore: Not enough space to cache rdd_263_0 in memory! (computed 19.7 MiB so far)
22/04/26 14:47:46 WARN MemoryStore: Not enough space to cache rdd_263_6 in memory! (computed 19.7 MiB so far)
22/04/26 14:47:46 WARN MemoryStore: Not enough space to cache rdd_26

In [66]:
type(model)

pyspark.ml.regression.RandomForestRegressionModel

### Model Testing 

In [67]:
df_pred = model.transform(flights_test)
df_pred.show()

+------+----+-------+--------+--------+------------------+
|Origin|Dest|AirTime|Distance|features|        prediction|
+------+----+-------+--------+--------+------------------+
|   ABE| CLT|     72|   774.0| [774.0]| 78.72463385463007|
|   ABE| CLT|     77|   774.0| [774.0]| 78.72463385463007|
|   ABE| CLT|     83|   774.0| [774.0]| 78.72463385463007|
|   ABE| CLT|     84|   774.0| [774.0]| 78.72463385463007|
|   ABE| CLT|     90|   774.0| [774.0]| 78.72463385463007|
|   ABE| PHL|     21|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     21|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     21|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     22|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     23|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     23|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     24|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     26|    89.0|  [89.0]|22.644514399204862|
|   ABE| PHL|     28|    89.0|  [89.0]|22.64451439920486

# View model summary

In [68]:
from pyspark.ml.evaluation import RegressionEvaluator

In [69]:
evaluator = RegressionEvaluator(labelCol="AirTime", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(df_pred)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 16.1997
